In [1]:
## Loading Libraries
import pandas as pd
import numpy as np
import sys
import jdatetime as jd
import os
import pickle

# import power_index_calculator as px
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
## Loading functions
from convert_ar_characters import convert_ar_characters
from ReadingBalancesheetData import read_blnc_data
from nthMax import nth_max
from gini import gini
from find_shapley import find_shapley
from find_banzhaf import find_banzhaf
from gameTheoric_concentration import gameTheoric_concentration
from fill_shapley_banzhaf import fill_shapley_banzhaf

# **Loading Shareholder Data**

In [3]:
## Loading DATA
os.chdir(r"C:\Users\Mahdi\OneDrive\Master Thesis\Data")
SDATA = pd.read_csv("Shareholder1398.csv",index_col=0)

# Conver date from string to jdatetime
SDATA['True_Date'] = pd.to_datetime(SDATA['True_Date'], format='%Y-%m-%d')
G = SDATA.True_Date.drop_duplicates()
J = G.apply(lambda x: jd.date.fromgregorian(day=x.day,month=x.month,year=x.year))
DataOrg_date_GtoJ_dict = dict(zip(G,J))
SDATA['Jalali_Date']=SDATA.True_Date.map(DataOrg_date_GtoJ_dict)

SDATA.drop(columns=['High', 'Low', 'Open', 'Last', 'Volume', 'close',
       'True_Date', 'year', 'month', 'day', 'Fill_Flag','Unadjusted_close','chnk_id'],inplace=True)

In [4]:
# Creating Dataframe for saving concentration mearsurs
CMdf = SDATA.groupby('Symbol',as_index=False).agg({'Id_tse':'first','percent':'sum','ShareHolder':'count',
                                                   'MarketCap':'first'}).rename(columns={'ShareHolder':'Num_holders',
                                                                                         'percent':'sum_over1'})
CMdf.reset_index(drop=True,inplace=True)

print('Number of Observation is: ',len(CMdf),'\n')

## Filtering Data
theta_sum1 = 90
theta_holder = 2
MC = 100e10


filtered = CMdf[CMdf.sum_over1>=theta_sum1]
CMdf = CMdf[CMdf.sum_over1<=theta_sum1]
print(len(filtered),' is deleted becaause their sum_above1% is equalt to or more than ',theta_sum1,'%')

filtered = CMdf[CMdf.Num_holders<=theta_holder]
CMdf = CMdf[CMdf.Num_holders>theta_holder]
print(len(filtered),' is deleted becaause their number of holder is less or equal than ',theta_holder)

filtered = CMdf[CMdf.MarketCap<=MC]
CMdf = CMdf[np.logical_or(CMdf.MarketCap>MC,pd.isnull(CMdf.MarketCap))]
print(len(filtered),' is deleted becaause their MarketCap is less or equal than ',MC)

print('\nNumber of remained Observation is: ',len(CMdf))

Number of Observation is:  655 

122  is deleted becaause their sum_above1% is equalt to or more than  90 %
159  is deleted becaause their number of holder is less or equal than  2
0  is deleted becaause their MarketCap is less or equal than  1000000000000.0

Number of remained Observation is:  374


# **Concentration Measures**

## 1- Largest Owner

In [5]:
temp = SDATA.groupby('Symbol',as_index=False).agg({'percent':'max'}).rename(columns={'percent':'Largest_Owner'})
CMdf = pd.merge(CMdf,temp,left_on='Symbol',right_on='Symbol',how='left')

## 2- First/Second

In [6]:
temp = SDATA.groupby('Symbol',as_index=False).agg({'percent':{lambda x: max(x)/nth_max(x,nth=2,interval=False)}}).rename(columns={'percent':'First_Second'})
CMdf = pd.merge(CMdf,temp,left_on='Symbol',right_on='Symbol',how='left').rename(columns={('First_Second', '<lambda>'):'First_Second'})

C:\Users\Mahdi\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:618: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
C:\Users\Mahdi\Anaconda3\lib\site-packages\pandas\core\generic.py:3936: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


## 3- First/Sumtwofour

In [7]:
temp = SDATA.groupby('Symbol',as_index=False).agg({'percent':{lambda x: max(x)/sum(nth_max(x,nth=[2,4],interval=True))}}).rename(
    columns={'percent':'First_Sumtwofour'})
CMdf = pd.merge(CMdf,temp,left_on='Symbol',right_on='Symbol',how='left').rename(columns={('First_Sumtwofour', '<lambda>'):'First_Sumtwofour'})

## 4- Sumfive

In [8]:
temp = SDATA.groupby('Symbol',as_index=False).agg({'percent':{lambda x: sum(nth_max(x,nth=[1,5],interval=True))}}).rename(columns={'percent':'Sumfive'})
CMdf = pd.merge(CMdf,temp,left_on='Symbol',right_on='Symbol',how='left').rename(columns={('Sumfive', '<lambda>'):'Sumfive'})

## 5- Gini

We calculate gini coeficient using Deaton 1997:
$$ \gamma = \frac{N+1}{N-1} - \frac{2}{\mu\times N\times(N-1)}\sum_{i=1}^N{\rho_ix_i}$$

In [9]:
temp = SDATA.groupby('Symbol',as_index=False).agg({'percent':{lambda x: gini(list(x))}}).rename(columns={'percent':'Gini'})
CMdf = pd.merge(CMdf,temp,left_on='Symbol',right_on='Symbol',how='left').rename(columns={('Gini', '<lambda>'):'Gini'})

## 6- Herfindhal

In [10]:
temp = SDATA.groupby('Symbol',as_index=False).agg({'percent':{lambda x: sum([(t/100)**2 for t in list(x)])}}).rename(columns={'percent':'Herfindhal'})
CMdf = pd.merge(CMdf,temp,left_on='Symbol',right_on='Symbol',how='left').rename(columns={('Herfindhal', '<lambda>'):'Herfindhal'})

## 7 & 8-Shapley-Shubik & Banzhaf

In [11]:
# # Initiating columns
# CMdf['SSCL'] = np.nan
# CMdf['SSCO'] = np.nan
# CMdf['SSDL'] = np.nan
# CMdf['SSDO'] = np.nan
# CMdf['BZCL'] = np.nan
# CMdf['BZCO'] = np.nan
# CMdf['BZDL'] = np.nan

os.chdir(r"C:\Users\Mahdi\OneDrive\Master Thesis\Data")
CMdf_load = pd.read_csv('Measures1398.csv',index_col=0)

CMdf = pd.merge(CMdf,CMdf_load[['Symbol','SSCL', 'SSCO', 'SSDL', 'SSDO', 'BZCL', 'BZCO', 'BZDL']],left_on='Symbol',right_on='Symbol',how='left')

In [ ]:
%%time
data = fill_shapley_banzhaf(data = CMdf,SDATA=SDATA,fast_mode = True,time_pnt=True,major_thr = 10)
CMdf = data['CMdf']

print('len(Errors): ',len(data['Errors']))
data['Errors']
[x for x in data['Errors'] if x[2]!= 'Error: request error!']

In [12]:
# CMdf.loc[CMdf['SSCO']==np.inf,'SSCO'] = np.nan
# CMdf.loc[CMdf['Num_holders']==1,'First_Second'] = np.nan
# CMdf.loc[CMdf['Num_holders']==2,'First_Sumtwofour'] = np.nan
# CMdf.loc[CMdf['Num_holders']==1,'Gini'] = np.nan

os.chdir(r"C:\Users\Mahdi\OneDrive\Master Thesis\Data")
CMdf.to_csv('Measures1398.csv')

# **Loading Balancesheet Data and Merging**

In [13]:
blnc_data = read_blnc_data(file='98.txt')
blnc_data['book_value'] = blnc_data.Tot_asset-blnc_data.Tot_lib
blnc_data = blnc_data[blnc_data.book_value>0]

In [14]:
data_out = pd.merge(CMdf,blnc_data[['Symbol','Tot_asset', 'Tot_lib', 'Capital', 'Equity_at_year_end','Debt_Equity', 'Debt_Equity_normal', 'book_value']]
                    ,left_on='Symbol',right_on='Symbol',how='outer')

os.chdir(r"C:\Users\Mahdi\OneDrive\Master Thesis\Data")
data_out.to_csv('data_owenership98_blnc98.csv')

In [25]:
np.round(CMdf.describe(),3).to_csv('OCM_Stat.csv')

In [24]:
np.round(CMdf.describe(),3)

,Id_tse,sum_over1,Num_holders,MarketCap,Largest_Owner,First_Second,First_Sumtwofour,Sumfive,Gini,Herfindhal,SSCL,SSCO,SSDL,SSDO,BZCL,BZCO,BZDL
count,3.740000e+02,374.000,374.000,5.300000e+01,374.000,374.000,374.000,374.000,374.000,374.000,374.000,364.000,367.000,367.000,374.000,374.000,374.000
mean,3.617226e+16,67.680,5.936,2.588607e+13,40.098,5.953,3.254,64.059,0.625,0.231,0.611,0.207,0.623,0.202,0.679,0.165,0.792
std,2.044339e+16,18.444,4.074,5.575595e+13,19.269,8.306,4.101,19.587,0.197,0.157,0.346,0.229,0.343,0.228,0.361,0.227,0.305
min,4.089344e+14,6.980,3.000,1.640811e+12,2.600,1.000,0.333,6.980,0.063,0.002,0.026,0.000,0.027,0.000,0.026,0.000,0.062
25%,1.809666e+16,57.732,4.000,7.049370e+12,22.577,1.620,0.880,52.440,0.497,0.100,0.271,0.000,0.288,0.000,0.318,0.000,0.500
50%,3.689308e+16,72.605,5.000,1.160100e+13,40.815,2.701,1.729,68.830,0.639,0.212,0.589,0.154,0.608,0.149,0.977,0.006,1.000
75%,5.274312e+16,81.773,7.000,2.815400e+13,52.637,6.194,3.771,79.747,0.789,0.316,1.000,0.317,1.000,0.312,1.000,0.285,1.000
max,7.174468e+16,89.990,38.000,3.944385e+14,85.090,59.343,28.747,89.990,0.955,0.725,1.000,0.929,1.000,0.928,1.000,0.928,1.000


In [27]:
CMdf[CMdf.SSDO>0.9]

,Symbol,Id_tse,sum_over1,Num_holders,MarketCap,Largest_Owner,First_Second,First_Sumtwofour,Sumfive,Gini,Herfindhal,SSCL,SSCO,SSDL,SSDO,BZCL,BZCO,BZDL
21,بمپنا,39807886630843041,7.59,4,NaN,2.60,1.268293,0.521042,7.59,0.185771,0.001532,0.026420,0.923518,0.026669,0.922627,0.026489,0.923344,0.5
269,وتوس,68117765376081366,6.98,3,NaN,3.85,2.264706,1.230032,6.98,0.346705,0.001976,0.039633,0.929069,0.040020,0.928240,0.040319,0.928318,1.0
